In [12]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import xgboost

In [2]:
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=20000)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

In [6]:
rfc = RandomForestClassifier(max_depth=5, n_jobs=-1).fit(X_train, y_train)
etc = ExtraTreesClassifier(max_depth=5, n_jobs=-1).fit(X_train, y_train)
svc = SVC(probability=True).fit(X_train, y_train)
accuracy_score(rfc.predict(X_val), y_val), accuracy_score(etc.predict(X_val), y_val), accuracy_score(svc.predict(X_val), y_val)

(0.8571, 0.8359, 0.977)

In [7]:
vch = VotingClassifier(estimators=[("rfc", rfc), ("etc", etc), ("svc", svc)], voting="hard", n_jobs=-1)
vcs = VotingClassifier(estimators=[("rfc", rfc), ("etc", etc), ("svc", svc)], voting="soft", n_jobs=-1)
vch.fit(X_train, y_train)
vcs.fit(X_train, y_train)
accuracy_score(vch.predict(X_val), y_val), accuracy_score(vcs.predict(X_val), y_val) 

(0.8802, 0.974)

# Mixer

In [17]:
layer1_data = np.concatenate([rfc.predict(X_val).reshape(-1,1), etc.predict(X_val).reshape(-1,1), svc.predict(X_val).reshape(-1,1)], axis=1)

/home/bartosz/anaconda3/envs/aurelion/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:09:22] WARNING: /croot/xgboost-split_1675119646044/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


ValueError: Feature shape mismatch, expected: 3, got 784

In [22]:
xgb = xgboost.XGBClassifier().fit(layer1_data, y_val)

/home/bartosz/anaconda3/envs/aurelion/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:15:44] WARNING: /croot/xgboost-split_1675119646044/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [23]:
layer1t_data = np.concatenate([rfc.predict(X_test).reshape(-1,1), etc.predict(X_test).reshape(-1,1), svc.predict(X_test).reshape(-1,1)], axis=1)

In [24]:
accuracy_score(xgb.predict(layer1t_data), y_test)

0.9792